---
title: Getting Started
html-table-processing: none
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.17.1
  kernelspec:
    display_name: .venv
    language: python
    name: python3
---

## Collections

A **collection** is the central unit here. It offers an object that allows you to create, access and query its entries, from now on denoted as a **Simulation**.

Collections are implicitally created if they don’t exist yet.
The first argument to [Collection](/apidocs/core/collection#Collection) is a path. Let's create our first collection at `./data-foo`.
This will create the directory and assign a unique ID to the new collection.


In [1]:
# Creating a new Collection
from bamboost import Collection

coll = Collection("data-foo")
coll

Database,/home/florez/work/code/docs/trial-quarto/data-foo
UID,9CA53B1CE7
Size,0


Once created, it is often easier and safer to use the uid to reference
the collection.

:::{.callout-note}

Although you can create and access collections with their path, it is good practice to explicitly create them using the [command line interface](cli.md) and then use their ID in your code.

```bash
bamboost-cli new ./data-foo
bamboost-cli list
```

:::

In [2]:
coll = Collection(uid="315628DE80")

If you are working in an ipython session (e.g. a jupyter notebook), use
[Collection.fromUID](/apidocs/core/collection#Collection) which will give you autocompletion for all
of your existing (cached) collections.

In [3]:
coll = Collection.fromUID["315628DE80 - ost-docs/content/docs/data-foo"]

## Simulations

Now that you have a collection, you can create simulations inside it.
A simulation not only stores the output of your experiment, but simultaneously acts as its input file (assuming you run numerical experiments).

This means, the same entity is used in multiple steps of your workflow; experimental design, execution, postprocessing/analysis.

### Experimental design

<Steps>

<Step>

#### Creation

</Step>

You have the intention of running a certain experiment with a specific set of input parameters, or input files, or anything.
So we create the simulation with all the instructions it needs.
Bundling all of this in a single place ensures reproducability.
This most likely includes:

- A dictionary of parameters
- A script that produces the result for this simulation
- A set of instructions on how to run the script

To create a new simulation, use [create_simulation](/apidocs/core/collection#Collection.create_simulation):

In [4]:
sim = coll.create_simulation(
    parameters={
        "param1": 73,
        "bar": [2, 3, 4, 5],
    }
)

<Step>

#### Relevant files

</Step>

Then, copy relevant files (or entire directories) into the simulation directory.

:::{.callout-note}
`create_simulation` includes a `files` argument to directly copy a list of files or directories.
:::

In [5]:
sim.copy_files(["path/to/script.py", "img1.png", "path/to/some/directory"])

<Step>

#### Run script

</Step>

As a next step we can create a run script for the simulation. This is an auto-generated bash script with the purpose of providing a single access point to produce the results for this simulation.

[`create_run_script`](/apidocs/core/simulation/base#_Simulation.create_run_script) takes up to 3 arguments:

- `commands`: an iterable of bash commands to run in sequence.
- `euler`: a boolean flag. If set to true, then a slurm submission script is written instead of a pure bash script.
- `sbatch_kwargs`: a dictionary of slurm job arguments.

In [6]:
sim.create_run_script(
    commands=["source .venv/bin/activate", "python3 script.py"], euler=False
)

Which will create the following file...

```{.bash filename="run.sh"}
#!/bin/bash

export SIMULATION_DIR=/home/florez/work/code/bamboost-docs/content/docs/data-foo/04d11573c0
export SIMULATION_ID=315628DE80:04d11573c0

source .venv/bin/activate
python3 script.py
```

Notice that the script exports two variables; `SIMULATION_DIR` and `SIMULATION_ID`. These should be used in your executable script `script.py` to infer the simulation on which to execute the script.

</Steps>

### Execution

After creation, a simulation is started by running it's run script. Of course, you could manually run it, however, you can also start it from within python or using the CLI (not implemented yet).

:::{.callout-note}
If you use the bamboost methods to submit your job, and you have previously created a SLURM submission script instead of a pure bash script, your job is automatically submitted on the cluster.
:::

:::{.callout-note}
You can also submit the simulation like this directly after you create it.
:::

```{.python filename="Submit simulation from python"}
sim.submit_simulation()
```

```{.bash filename="Submit simulation using CLI"}
bamboost-cli Collection-ID Simulation-Name
```

### Postprocessing

After your job has finished, it's time to mine the diamonds.
Besides the workflow management described in this document, `bamboost` also provides a data model using a HDF5 file (see the docs on how to [write data using bamboost](writing)).
Much of the postprocessing capabilities depend on using the bamboost hdf5 module, but some is also general.

:::{.callout-note}
`bamboost` is optimized for interactive use in jupyter notebooks. It offers autocompletion for most of its objects, such as collections and simulation, but also objects and data that is part of your simulations.
:::

<Steps>

<Step>

#### Collection

</Step>

Initialize the collection as seen above, preferably using it's ID.
[coll.df](/apidocs/core/collection#Collection.df) returns a pandas dataframe of the collection.

In [7]:
coll = Collection.fromUID["315628DE80"]
coll.df

,name,created_at,description,status,submitted,bar,param1
0,8750a0602c,2025-05-08 05:00:18.754193,,initialized,False,"[2, 3, 4, 5]",73
1,dbd3875c4d,2025-05-08 04:55:54.099025,,initialized,False,"[2, 3, 4, 5]",73
2,bca31949da,2025-05-08 04:55:14.134284,,initialized,False,"[2, 3, 4, 5]",73
3,be5f3d8d2a,2025-05-08 04:54:46.522284,,initialized,False,"[2, 3, 4, 5]",73
4,9a4ed926eb,2025-05-08 04:53:19.296997,,initialized,False,"[2, 3, 4, 5]",73
5,4432041ff2,2025-05-08 04:40:45.435123,,initialized,False,"[2, 3, 4, 5]",73
6,8c0a09e749,2025-05-08 04:37:43.293220,,initialized,False,"[2, 3, 4, 5]",73
7,f1ab07bd78,2025-05-08 04:35:48.146220,,initialized,False,"[2, 3, 4, 5]",73
8,7fbe18faf1,2025-05-08 04:34:51.688932,,initialized,False,"[2, 3, 4, 5]",73
9,6bdf766aa1,2025-05-08 04:26:18.117418,,initialized,False,"[2, 3, 4, 5]",73


For now, you can use the dataframe to query simulations based on their parameters.

<Callout type="warn">
The custom filtering interface is currently work in progress
</Callout>

<Step>

#### Simulation

</Step>

To get a [Simulation](/apidocs/core/simulation/base/Simulation) object, use brackets (it will autocomplete in notebooks).

In [8]:
sim = coll["04d11573c0"]
sim

Parameter,Value
bar,[2 3 4 5]
param1,73


A few things the simulation object offers are:

- `sim.files`: A file picker that returns the absolute path of files stored in this simulation
- `sim.parameters`: The parameters
- `sim.metadata`: some metadata
- `sim.root`: Access to the root group inside `data.h5` (more on the `h5py` wrapper of `bamboost` will be written elsewhere).
- `sim.data`: The main (time)-series of the simulation
- much more

</Steps>

To be continued...